## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**3521. Find Product Recommendation Pairs (Medium)**

**Table: ProductPurchases**

| Column Name | Type | 
|-------------|------|
| user_id     | int  |
| product_id  | int  |
| quantity    | int  |

(user_id, product_id) is the unique key for this table.
Each row represents a purchase of a product by a user in a specific quantity.
Table: ProductInfo

| Column Name | Type    | 
|-------------|---------|
| product_id  | int     |
| category    | varchar |
| price       | decimal |

product_id is the primary key for this table.
Each row assigns a category and price to a product.

Amazon wants to implement the **Customers who bought this also bought...** feature based on **co-purchase patterns**. Write a solution to :

1. Identify **distinct** product pairs frequently **purchased together by the same customers** (where **`product1_id`** < **`product2_id`**)
2. For **each product pair**, determine how many customers purchased **both** products

**A product pair** is considered for recommendation **if at least** **`3`** **different** customers have purchased **both products**.

Return the result table ordered by **customer_count** in **descending** order, and in case of a tie, by **`product1_id`** in **ascending** order, and then by **`product2_id`** in **ascending** order.

The result format is in the following example.

**Example:**

**Input:**

**ProductPurchases table:**

| user_id | product_id | quantity |
|---------|------------|----------|
| 1       | 101        | 2        |
| 1       | 102        | 1        |
| 1       | 103        | 3        |
| 2       | 101        | 1        |
| 2       | 102        | 5        |
| 2       | 104        | 1        |
| 3       | 101        | 2        |
| 3       | 103        | 1        |
| 3       | 105        | 4        |
| 4       | 101        | 1        |
| 4       | 102        | 1        |
| 4       | 103        | 2        |
| 4       | 104        | 3        |
| 5       | 102        | 2        |
| 5       | 104        | 1        |


**ProductInfo table:**

| product_id | category    | price |
|------------|-------------|-------|
| 101        | Electronics | 100   |
| 102        | Books       | 20    |
| 103        | Clothing    | 35    |
| 104        | Kitchen     | 50    |
| 105        | Sports      | 75    |


**Output:**

| product1_id | product2_id | product1_category | product2_category | customer_count |
|-------------|-------------|-------------------|-------------------|----------------|
| 101         | 102         | Electronics       | Books             | 3              |
| 101         | 103         | Electronics       | Clothing          | 3              |
| 102         | 104         | Books             | Kitchen           | 3              |


**Explanation:**
- **Product pair (101, 102):**
    - Purchased by users 1, 2, and 4 (3 customers)
    - Product 101 is in Electronics category
    - Product 102 is in Books category
- **Product pair (101, 103):**
    - Purchased by users 1, 3, and 4 (3 customers)
    - Product 101 is in Electronics category
    - Product 103 is in Clothing category
- **Product pair (102, 104):**
    - Purchased by users 2, 4, and 5 (3 customers)
    - Product 102 is in Books category
    - Product 104 is in Kitchen category

The result is ordered by customer_count in descending order. For pairs with the same customer_count, they are ordered by product1_id and then product2_id in ascending order

In [0]:
purchases_data_3521 = [
    (1, 101, 2),
    (1, 102, 1),
    (1, 103, 3),
    (2, 101, 1),
    (2, 102, 5),
    (2, 104, 1),
    (3, 101, 2),
    (3, 103, 1),
    (3, 105, 4),
    (4, 101, 1),
    (4, 102, 1),
    (4, 103, 2),
    (4, 104, 3),
    (5, 102, 2),
    (5, 104, 1)
]

purchases_columns_3521 = ["user_id", "product_id", "quantity"]

purchases_df_3521 = spark.createDataFrame(purchases_data_3521, purchases_columns_3521)

product_info_data_3521 = [
    (101, "Electronics", 100),
    (102, "Books", 20),
    (103, "Clothing", 35),
    (104, "Kitchen", 50),
    (105, "Sports", 75)
]

product_info_columns_3521 = ["product_id", "category", "price"]

product_info_df_3521 = spark.createDataFrame(product_info_data_3521, product_info_columns_3521)

purchases_df_3521.show()
product_info_df_3521.show()


+-------+----------+--------+
|user_id|product_id|quantity|
+-------+----------+--------+
|      1|       101|       2|
|      1|       102|       1|
|      1|       103|       3|
|      2|       101|       1|
|      2|       102|       5|
|      2|       104|       1|
|      3|       101|       2|
|      3|       103|       1|
|      3|       105|       4|
|      4|       101|       1|
|      4|       102|       1|
|      4|       103|       2|
|      4|       104|       3|
|      5|       102|       2|
|      5|       104|       1|
+-------+----------+--------+

+----------+-----------+-----+
|product_id|   category|price|
+----------+-----------+-----+
|       101|Electronics|  100|
|       102|      Books|   20|
|       103|   Clothing|   35|
|       104|    Kitchen|   50|
|       105|     Sports|   75|
+----------+-----------+-----+



In [0]:
pairs_df_3521 = purchases_df_3521.alias("p1")\
                    .join( purchases_df_3521.alias("p2"),
                            (col("p1.user_id") == col("p2.user_id")) &
                            (col("p1.product_id") < col("p2.product_id"))
                        )\
                        .select(
                                col("p1.product_id").alias("product1_id"),
                                col("p2.product_id").alias("product2_id"),
                                col("p1.user_id").alias("user_id")
                                )

In [0]:
pair_counts_3521 = pairs_df_3521\
                        .groupBy("product1_id", "product2_id")\
                            .agg(
                                countDistinct("user_id").alias("customer_count")
                                ).filter(col("customer_count") >= 3)

In [0]:
pair_counts_3521\
    .join(
        product_info_df_3521.alias("pi1"),
        col("product1_id") == col("pi1.product_id")
    )\
        .join(
        product_info_df_3521.alias("pi2"),
        col("product2_id") == col("pi2.product_id")
        )\
        .select(
            "product1_id",
            "product2_id",
            col("pi1.category").alias("product1_category"),
            col("pi2.category").alias("product2_category"),
            "customer_count"
        )\
        .orderBy(
            col("customer_count").desc(),
            col("product1_id").asc(),
            col("product2_id").asc()
    ).display()

product1_id,product2_id,product1_category,product2_category,customer_count
101,102,Electronics,Books,3
101,103,Electronics,Clothing,3
102,104,Books,Kitchen,3
